In [5]:
from biopandas.pdb import PandasPdb
import pandas as pd
import nglview as ngv
from collections import Counter
import seaborn as sns

In [6]:
model0 = PandasPdb().read_pdb('model0_chainID.pdb')
ngv.show_file('model0_chainID.pdb')

NGLWidget()

In [7]:
df = model0.df['ATOM']
df

,record_name,atom_number,blank_1,atom_name,alt_loc,residue_name,blank_2,chain_id,residue_number,insertion,...,x_coord,y_coord,z_coord,occupancy,b_factor,blank_4,segment_id,element_symbol,charge,line_idx
0,ATOM,1,,N,,TYR,,A,1,,...,142.54,145.02,5.97,1.0,0.0,,,N,NaN,5
1,ATOM,2,,H1,,TYR,,A,1,,...,142.68,144.17,6.49,1.0,0.0,,,H,NaN,6
2,ATOM,3,,H2,,TYR,,A,1,,...,142.77,145.76,6.63,1.0,0.0,,,H,NaN,7
3,ATOM,4,,H3,,TYR,,A,1,,...,143.14,144.90,5.17,1.0,0.0,,,H,NaN,8
4,ATOM,5,,CA,,TYR,,A,1,,...,141.15,144.93,5.52,1.0,0.0,,,C,NaN,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69067,ATOM,69068,,H4,,CAP,,8,465,,...,151.47,108.04,80.71,1.0,0.0,,,H,NaN,69072
69068,ATOM,69069,,H12,,CAP,,8,465,,...,152.99,109.29,80.42,1.0,0.0,,,H,NaN,69073
69069,ATOM,69070,,H11,,CAP,,8,465,,...,153.11,110.15,78.93,1.0,0.0,,,H,NaN,69074
69070,ATOM,69071,,HO4,,CAP,,8,465,,...,150.90,106.88,78.63,1.0,0.0,,,H,NaN,69075


In [8]:
LSU = ['A','C','E','G','I','K','M','O'] #large subunit
SSU = ['B','D','F','H','J','L','N','P'] #small subunit
chains = ['A','B','C','D','E','F','G','H','I','J','K','L','M','N','O','P']
PASR_name=['GLU','ASP','KCX','HID','HIE','HIP','ARG','LYS']

#protonation states of each PASR_name
acid = {'GLU-':['N','H','CA','HA','CB','HB2','HB3','CG','HG2','HG3','CD','OE1','OE2','C','O'],
        'GLU0':['N','H','CA','HA','CB','HB2','HB3','CG','HG2','HG3','CD','OE1','OE2','HE2','C','O'],
        'KCX-':['N','CA','CB','CG','CD','CE','NZ','C','O','CX','OQ1','OQ2','H13','H14','H15','H16','H3','H5','H6','H20','H21','H22', 'H23'],
        'KCX0':['N','CA','CB','CG','CD','CE','NZ','C','O','CX','OQ1','OQ2','H13','H14','H15','H16','H3','H5','H6','H20','H21','H22','H23','H27'],
        'ASP-':['N','H','CA','HA','CB','HB2','HB3','CG','OD1','OD2','C','O'],
        'ASP0':['N','H','CA','HA','CB','HB2','HB3','CG','OD1','OD2','HD2','C','O']}
basic = {'LYS+':['N','H','CA','HA','CB','HB2','HB3','CG','HG2','HG3','CD','HD2','HD3','CE','HE2','HE3','NZ','HZ1','HZ2','HZ3','C','O'], 
         'LYS0':['N','H','CA','HA','CB','HB2','HB3','CG','HG2','HG3','CD','HD2','HD3','CE','HE2','HE3','NZ','HZ1','HZ2','C','O'],
         'HIP+':['N','H','CA','HA','CB','HB2','HB3','CG','ND1','HD1','CE1','HE1','NE2','HE2','CD2','HD2','C','O'],
         'HIE0':['N','H','CA','HA','CB','HB2','HB3','CG','ND1','CE1','HE1','NE2','HE2','CD2','HD2','C','O'],
         'HID0':['N','H','CA','HA','CB','HB2','HB3','CG','ND1','HD1','CE1','HE1','NE2','CD2','HD2','C','O'],         
         'ARG+':['N','H','CA','HA','CB','HB2','HB3','CG','HG2','HG3','CD','HD2','HD3','NE','HE','CZ','NH1','1HH1','2HH1','NH2','1HH2','2HH2','C','O'],
         'ARG0':['N','H','CA','HA','CB','HB2','HB3','CG','HG2','HG3','CD','HD2','HD3','NE','HE','CZ','NH1','2HH1','NH2','1HH2','2HH2','C','O']}

In [9]:
acid_base = [] #A, B
charge = [] #1, 0, -1
name = []
id_number = []
chain_id = []

for chain in chains:
    if chain in LSU:
        for res in range(1,451):
            residue_df = df.loc[(df['chain_id']==chain)&(df['residue_number']==res)]
            residue_name = residue_df['residue_name'].unique()
            cntr = Counter(residue_df['atom_name'])
    
            if residue_name in ['GLU','ASP','KCX']: #acid
                if cntr == Counter(acid[residue_name[0]+'0']):
                    #print(chain,res,residue_name,'acid', 'neutral')
                    acid_base.append('A')
                    charge.append(0)
                    name.append(residue_name)
                    id_number.append(res)
                    chain_id.append(chain)
                    
                    
                elif cntr == Counter(acid[residue_name[0]+'-']):
                    #print(chain,res,residue_name,'acid', 'negative')
                    acid_base.append('A')
                    charge.append(-1)
                    name.append(residue_name)
                    id_number.append(res)
                    chain_id.append(chain)
                    
                else:
                    print('Error_Acid')
                    
            if residue_name in ['LYS', 'ARG']: #basic
                if cntr == Counter(basic[residue_name[0]+'0']):
                    #print(chain,res,residue_name,'basic', 'neutral')
                    acid_base.append('B')
                    charge.append(0)
                    name.append(residue_name)
                    id_number.append(res)
                    chain_id.append(chain)
                    
                elif cntr == Counter(basic[residue_name[0]+'+']):
                    #print(chain,res,residue_name,'basic', 'positive')
                    acid_base.append('B')
                    charge.append(1)
                    name.append(residue_name)
                    id_number.append(res)
                    chain_id.append(chain)
                    
                else:
                    print('Error_Base')
                    
            if residue_name in ['HIE', 'HID', 'HIP']: #histidines
                if cntr == Counter(basic[residue_name[0]+'0']):
                    #print(chain,res,residue_name,'basic', 'neutral')
                    acid_base.append('B')
                    charge.append(0)
                    name.append(residue_name)
                    id_number.append(res)
                    chain_id.append(chain)

                    
                elif cntr == Counter(basic['HIP'+'+']):
                    #print(chain,res,residue_name,'basic', 'positive')
                    acid_base.append('B')
                    charge.append(1)
                    name.append(residue_name)
                    id_number.append(res)
                    chain_id.append(chain)
                    
                else:
                    print('Error_Base')
   
    if chain in SSU:
        for res in range(1,100):
            residue_df = df.loc[(df['chain_id']==chain)&(df['residue_number']==res)]
            residue_name = residue_df['residue_name'].unique()
            cntr = Counter(residue_df['atom_name'])
          
            if residue_name in ['GLU','ASP','KCX']: #acid
                if cntr == Counter(acid[residue_name[0]+'0']):
                    #print(chain,res,residue_name,'acid', 'neutral')
                    acid_base.append('A')
                    charge.append(0)
                    name.append(residue_name)
                    id_number.append(res)
                    chain_id.append(chain)
                    
                elif cntr == Counter(acid[residue_name[0]+'-']):
                    #print(chain,res,residue_name,'acid', 'negative')
                    acid_base.append('A')
                    charge.append(-1)
                    name.append(residue_name)
                    id_number.append(res)
                    chain_id.append(chain)
                    
                else:
                    print('Error_Acid')
                    
            if residue_name in ['LYS', 'ARG']: #basic
                if cntr == Counter(basic[residue_name[0]+'0']):
                    #print(chain,res,residue_name,'basic', 'neutral')
                    acid_base.append('B')
                    charge.append(0)
                    name.append(residue_name)
                    id_number.append(res)
                    chain_id.append(chain)

                    
                elif cntr == Counter(basic[residue_name[0]+'+']):
                    #print(chain,res,residue_name,'basic', 'positive')
                    acid_base.append('B')
                    charge.append(1)
                    name.append(residue_name)
                    id_number.append(res)
                    chain_id.append(chain)
                    
                else:
                    print('Error_Base')
                    
            if residue_name in ['HIE', 'HID', 'HIP']: #histidines
                if cntr == Counter(basic[residue_name[0]+'0']):
                    #print(chain,res,residue_name,'basic', 'neutral')
                    acid_base.append('B')
                    charge.append(0)
                    name.append(residue_name)
                    id_number.append(res)
                    chain_id.append(chain)
                    
                elif cntr == Counter(basic['HIP'+'+']):
                    #print(chain,res,residue_name,'basic', 'positive')
                    acid_base.append('B')
                    charge.append(1)
                    name.append(residue_name)
                    id_number.append(res)
                    chain_id.append(chain)
                    
                else:
                    print('Error_HIS')

In [10]:
d = {'chain_id': chain_id, 'res_id': id_number, 'res_name':name,'acid/base': acid_base, 'charge': charge}
data = pd.DataFrame(data=d)
data

,chain_id,res_id,res_name,acid/base,charge
0,A,2,[GLU],A,-1
1,A,6,[ARG],B,1
2,A,9,[ARG],B,1
3,A,10,[GLU],A,-1
4,A,14,[ASP],A,-1
...,...,...,...,...,...
1275,P,75,[ASP],A,-1
1276,P,79,[LYS],B,1
1277,P,84,[ASP],A,-1
1278,P,87,[ARG],B,1


In [11]:
data.to_excel('protonation.xlsx')